<a href="https://colab.research.google.com/github/roostico/NesGen/blob/main/NESGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the dataset

In [ ]:
#@title Get the full version of the Lakh MIDI Dataset v0.1
!wget http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
!tar xvf lmd_full.tar.gz
!rm lmd_full.tar.gz

dataset_path = "/content/lmd_full"

In [2]:
#@title Get a smaller version of the Lakh MIDI Dataset v0.1
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar xvf clean_midi.tar.gz
!rm clean_midi.tar.gz

dataset_path = "/content/clean_midi"

Output streaming troncato alle ultime 5000 righe.
clean_midi/Camaleonti/
clean_midi/Camaleonti/Eternita.mid
clean_midi/Camaleonti/Applausi.1.mid
clean_midi/Camaleonti/L'ora dell'amore.1.mid
clean_midi/Camaleonti/L'ora dell'amore.2.mid
clean_midi/Camaleonti/L'ora dell'amore.3.mid
clean_midi/Camaleonti/Mamma mia.2.mid
clean_midi/Camaleonti/Viso d'angelo.mid
clean_midi/Camaleonti/Io per lei.mid
clean_midi/Camaleonti/Mamma mia.mid
clean_midi/Camaleonti/Perche ti amo.1.mid
clean_midi/Camaleonti/Perche ti amo.mid
clean_midi/Camaleonti/L'ora dell'amore.mid
clean_midi/Camaleonti/Viso d'angelo.1.mid
clean_midi/Camaleonti/Applausi.mid
clean_midi/Camaleonti/Mamma mia.1.mid
clean_midi/Camaleonti/Cuore di vetro.mid
clean_midi/Mike + The Mechanics/
clean_midi/Mike + The Mechanics/Silent Running.mid
clean_midi/Mike + The Mechanics/Another Cup of Coffee.2.mid
clean_midi/Mike + The Mechanics/Over My Schoulder.mid
clean_midi/Mike + The Mechanics/All I Need Is a Miracle.mid
clean_midi/Mike + The Mechanic

In [ ]:
#@title Get the NESMDB dataset
!gdown 1gIli7G1wu0QWDLzRc-CPWB8C4Hu0XVn3
!unzip nesmdb_midi.zip
!rm nesmdb_midi.zip

# Tokenization

## Installing MidiTok

In [3]:
!pip install miditok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.8 MB/s eta 0:00:00


## Setting up the tokenizer

In [4]:
from miditok import REMI, TokenizerConfig  # here we choose to use REMI
from pathlib import Path

# Creates the tokenizer and list the file paths
tokenizer = REMI()  # using defaults parameters (constants.py)


In [ ]:
!pip install miditoolkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [4]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=156360e13eec207b70f995ae29759acc5476bd3585fd2f29e4e5441bd34b9bc1
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


# Utility functions

In [46]:
def normalize_to_range(arr, range_min=0, range_max=1):
    # Get the minimum and maximum of the array
    arr_min = np.min(arr)
    arr_max = np.max(arr)

    # Normalize to [0, 1]
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)

    # Scale to [range_min, range_max] -> [-1, 1]
    scaled_arr = normalized_arr * (range_max - range_min) + range_min

    return scaled_arr, arr_min, arr_max

def de_normalize(arr, original_min, original_max, range_min=0, range_max=1):
    # Scale to [0, 1]
    scaled_arr = (arr - range_min) / (range_max - range_min)

    # Normalize to [original_min, original_max]
    de_normalized_arr = scaled_arr * (original_max - original_min) + original_min

    return de_normalized_arr

# Creation of the model

In [61]:
import os
import pretty_midi
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

def load_and_tokenize_midi(midi_files):
  tokenized_data = []
  for file in midi_files:
    try:
      midi_data = pretty_midi.PrettyMIDI(file)

      tokens = []
      for instrument in midi_data.instruments:
        for note in instrument.notes:
          tokens.append(note.pitch)

      tokenized_data.append(tokens)
      print(f"File {file} tokenizzato con successo.")
    except (OSError, Exception) as e:
            print(f"Errore nel caricamento del file {file}: {e}")
  return tokenized_data


def pad_sequences(sequences, maxlen=None):
    return tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')

def create_autoencoder(input_shape):
    """Creates an autoencoder model."""
    # Define input layer
    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    encoded = tf.keras.layers.LSTM(128, activation='relu')(input_layer)  # Modify units as needed

    # Decoder
    decoded = tf.keras.layers.RepeatVector(input_shape[0])(encoded)  # Use input_shape[0]
    decoded = tf.keras.layers.LSTM(input_shape[1], activation='relu', return_sequences=True)(decoded) # Use input_shape[1]

    # Create the autoencoder model
    autoencoder = tf.keras.Model(input_layer, decoded)

    # Compile the model
    autoencoder.compile(optimizer='adam', loss='mse')  # Consider other loss functions
    return autoencoder

def generate_midi(autoencoder, original_min, original_max, num_files=1):
    for _ in range(num_files):
        # Genera un vettore latente casuale
        random_latent_vector = np.random.normal(size=(1, 12800))  # Dimensione latente

        generated_sequence = autoencoder.predict(random_latent_vector)
        print(generated_sequence)
        generated_sequence = de_normalize(generated_sequence, original_min, original_max)
        # generated_sequence = np.argmax(generated_sequence, axis=-1)
        generated_sequence = np.squeeze(generated_sequence)
        generated_sequence = generated_sequence.astype(int)
        print(generated_sequence)
        print(generated_sequence.shape)

        generated_midi = pretty_midi.PrettyMIDI()
        instrument = pretty_midi.Instrument(program=0)

        for time_step in range(generated_sequence.shape[0]):
            pitch = generated_sequence[time_step]
            if pitch > 0:  # Evita il token 0, se usato per il padding
                note = pretty_midi.Note(velocity=100, pitch=pitch, start=time_step * 0.1, end=(time_step + 1) * 0.1)
                instrument.notes.append(note)

        generated_midi.instruments.append(instrument)
        generated_midi.write(f'generated_midi_{_}.mid')

# Training of the model

In [6]:
# Passo 0: Imposta il tuo flusso completo
midi_directory = dataset_path
midi_files = []

for root, _, files in os.walk(midi_directory):
    for file in files:
        if file.endswith('.mid'):
            midi_files.append(os.path.join(root, file))

In [7]:
# 1
tokenized_data = load_and_tokenize_midi(midi_files[:50])

Errore nel caricamento del file /content/clean_midi/Scott McKenzie/Forrest Gump: San Francisco (Be Sure to Wear Some Flowers in Your Hair).1.mid: data byte must be in range 0..127
File /content/clean_midi/Scott McKenzie/Forrest Gump: San Francisco (Be Sure to Wear Some Flowers in Your Hair).2.mid tokenizzato con successo.


/usr/local/lib/python3.10/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


File /content/clean_midi/Scott McKenzie/Forrest Gump: San Francisco (Be Sure to Wear Some Flowers in Your Hair).mid tokenizzato con successo.
File /content/clean_midi/Malmsteen Yngwie/Leviathan.mid tokenizzato con successo.
File /content/clean_midi/Malmsteen Yngwie/Crying.mid tokenizzato con successo.
File /content/clean_midi/The Marshall Tucker Band/Heard It in a Love Song.mid tokenizzato con successo.
File /content/clean_midi/The Cranberries/Will You Remember?.mid tokenizzato con successo.
File /content/clean_midi/The Cranberries/Not Sorry.mid tokenizzato con successo.
Errore nel caricamento del file /content/clean_midi/The Cranberries/Animal Instinct.mid: data byte must be in range 0..127
File /content/clean_midi/The Cranberries/How.mid tokenizzato con successo.
File /content/clean_midi/The Cranberries/Zombie.2.mid tokenizzato con successo.
File /content/clean_midi/The Cranberries/Hollywood.mid tokenizzato con successo.
File /content/clean_midi/The Cranberries/Ode to My Family.2.mid

In [8]:
# 2
max_sequence_length = max(len(seq) for seq in tokenized_data)  # Calcola la lunghezza massima
padded_data = pad_sequences(tokenized_data, maxlen=max_sequence_length)

In [31]:
# 3
input_data = np.array(padded_data)

import numpy as np

assert not np.any(np.isnan(input_data))
normalized_x, original_min, original_max = normalize_to_range(input_data, 0, 1)
assert (np.max(normalized_x)) == 1
assert (np.min(normalized_x)) == 0
print(normalized_x)


[[0.69607843 0.70588235 0.69607843 ... 0.         0.         0.        ]
 [0.39215686 0.39215686 0.39215686 ... 0.         0.         0.        ]
 [0.35294118 0.48039216 0.35294118 ... 0.         0.         0.        ]
 ...
 [0.34313725 0.55882353 0.45098039 ... 0.         0.         0.        ]
 [0.30392157 0.30392157 0.30392157 ... 0.         0.         0.        ]
 [0.49019608 0.57843137 0.53921569 ... 0.         0.         0.        ]]


In [17]:
autoencoder = create_autoencoder((normalized_x.shape[1], 1))

y = np.roll(normalized_x, shift=-1, axis=1)
reshaped_x = normalized_x.reshape(normalized_x.shape[0], normalized_x.shape[1], 1)
reshaped_y = y.reshape(y.shape[0], y.shape[1], 1)
print(reshaped_x)
print(reshaped_y)

[[[0.69607843]
  [0.70588235]
  [0.69607843]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.39215686]
  [0.39215686]
  [0.39215686]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.35294118]
  [0.48039216]
  [0.35294118]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 ...

 [[0.34313725]
  [0.55882353]
  [0.45098039]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.30392157]
  [0.30392157]
  [0.30392157]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.49019608]
  [0.57843137]
  [0.53921569]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]]
[[[0.70588235]
  [0.69607843]
  [0.69607843]
  ...
  [0.        ]
  [0.        ]
  [0.69607843]]

 [[0.39215686]
  [0.39215686]
  [0.37254902]
  ...
  [0.        ]
  [0.        ]
  [0.39215686]]

 [[0.48039216]
  [0.35294118]
  [0.5       ]
  ...
  [0.        ]
  [0.        ]
  [0.35294118]]

 ...

 [[0.55882353]
  [0.45098039]
  [0.34313725]
  ...
  [0.        ]
  [0.        ]
  [0.34313725]]

 [[0.303

In [18]:
# 4
autoencoder.fit(reshaped_x,
                 reshaped_y,  # Same data for autoencoder input and output
                 epochs=10,
                 batch_size=32)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 4s/step - loss: 0.1376
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - loss: 0.1348
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - loss: 0.1300
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.1230
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.1187
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.1068
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.0928
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.0822
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.0810
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - loss: 0.0795


In [62]:
generate_midi(autoencoder, num_files=1, original_min=original_min, original_max=original_max)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
[[[0.1143358 ]
  [0.19844712]
  [0.25868213]
  ...
  [0.39076546]
  [0.39076546]
  [0.39076546]]]
[11 20 26 ... 39 39 39]
(12800,)


# Utility functions

In [20]:
def random_file(root, keyword=None):
    import glob
    import os
    import random
    mid_files = glob.glob(os.path.join(root, "**", "*.mid"), recursive=True)
    if keyword is not None:
      mid_files = [file for file in mid_files if keyword in file.lower()]
    return random.choice(mid_files)

def generate_midi_from_tokens(tokens, tokenizer, output_path):
  from pathlib import Path
  # Convert to MIDI and save it
  generated_midi = tokenizer(tokens)  # MidiTok can handle PyTorch/Numpy/Tensorflow tensors
  generated_midi.dump_midi(Path(output_path))

# MIDI playing

## Installing the required libraries

In [21]:
!apt-get update -qq && apt-get install -y fluidsynth
!pip install pretty_midi midi-clip

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin libinput10
  libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5 libqt5gui5 libqt5network5
  libqt5svg5 libqt5widgets5 libwacom-bin libwacom-common libwacom9 libxcb-icccm4 libxcb-image0
  libxcb-keysyms1 libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qsynth qt5-gtk-platformtheme qttranslations5-l10n timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs qt5-image-formats-plugins qtwayland5 jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libevdev2 libfluidsynth3 libgudev-1.0-0 lib

## Download example Soundfonts (GeneralUser GS v2 and PICONICA)

In [22]:
!gdown 1wlpTIS70nQHMrYBjDT0M6nyg07kUejUv
!unzip GeneralUser_GS_v2.0.0--doc_r2.zip
!rm -rf GeneralUser_GS_v2.0.0--doc_r2.zip support documentation demo\ MIDIs
!mv GeneralUser\ GS\ v2.0.0.sf2 guGS.sf2

# PICONICA
!gdown 1uk51T9Gvo1n2JRl3_CHCg2FVGWiNI4qJ

Downloading...
From (original): https://drive.google.com/uc?id=1wlpTIS70nQHMrYBjDT0M6nyg07kUejUv
From (redirected): https://drive.google.com/uc?id=1wlpTIS70nQHMrYBjDT0M6nyg07kUejUv&confirm=t&uuid=c776058e-ba21-4d6a-85f7-a47aad9232a4
To: /content/GeneralUser_GS_v2.0.0--doc_r2.zip
100% 62.5M/62.5M [00:01<00:00, 57.0MB/s]
Archive:  GeneralUser_GS_v2.0.0--doc_r2.zip
   creating: demo MIDIs/
   creating: demo MIDIs/audio/
  inflating: demo MIDIs/audio/Bond.ogg  
  inflating: demo MIDIs/audio/Breakout.ogg  
  inflating: demo MIDIs/audio/Dance.ogg  
  inflating: demo MIDIs/audio/Earth Day - by Richard Audd (fixed).ogg  
  inflating: demo MIDIs/audio/J-cycle.ogg  
  inflating: demo MIDIs/audio/Jump!.ogg  
  inflating: demo MIDIs/audio/Santa Claus is Comin' to Town.ogg  
  inflating: demo MIDIs/audio/The HYBRID Collage (v2.0) - by S. Christian Collins.ogg  
  inflating: demo MIDIs/audio/Umi no Mieru Machi.ogg  
  inflating: demo MIDIs/Bond.mid     
  inflating: demo MIDIs/Breakout.mid  
  infla

## Optional: download other soundfonts

In [ ]:
# Pokemon
!gdown 1vDK_xH7WeAqQrrBFXfh4Q205x6oNhTQt

## Utility function to generate the audio on Colab

### Taken from https://github.com/bzamecnik/midi2audio/blob/master/midi2audio.py

In [23]:
import argparse
import os
import subprocess

__all__ = ['FluidSynth']

DEFAULT_SOUND_FONT = '~/.fluidsynth/default_sound_font.sf2'
DEFAULT_SAMPLE_RATE = 44100
DEFAULT_GAIN = 0.2

class FluidSynth():
    def __init__(self, sound_font=DEFAULT_SOUND_FONT, sample_rate=DEFAULT_SAMPLE_RATE, gain=DEFAULT_GAIN):
        self.sample_rate = sample_rate
        self.sound_font = os.path.expanduser(sound_font)
        self.gain = gain

    def midi_to_audio(self, midi_file: str, audio_file: str, verbose=True):
        if verbose:
            stdout = None
        else:
            stdout = subprocess.DEVNULL
        subprocess.call(
            ['fluidsynth', '-ni', '-g', str(self.gain), self.sound_font, midi_file, '-F', audio_file, '-r', str(self.sample_rate)],
            stdout=stdout,
        )

    def play_midi(self, midi_file):
        subprocess.call(['fluidsynth', '-i', '-g', str(self.gain), self.sound_font, midi_file, '-r', str(self.sample_rate)])

### Other utility functions

In [24]:
import pretty_midi
import os
import librosa.display

def show_midi_info(midi_path, print_notes=False):
  midi_data = pretty_midi.PrettyMIDI(midi_path)
  print("Instruments: ", [instrument.name for instrument in midi_data.instruments])
  print("MIDI duration: {duration:.2f} seconds".format(duration=midi_data.get_end_time()))
  if print_notes:
    for instrument in midi_data.instruments:
      print(instrument.name)
      for note in instrument.notes:
        print(note.start, note.end, note.pitch, note.velocity)

def piano_roll(midi_path):
  plt.figure(figsize=(12, 4))
  plot_piano_roll(path, 24, 84)

def plot_piano_roll(path, start_pitch, end_pitch, fs=100):
    midi_data = pretty_midi.PrettyMIDI(path)
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(midi_data.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))

def change_midi_velocity(midi_path, output_path, delta=0): # Renamed the function to avoid name conflict
  midi_data = pretty_midi.PrettyMIDI(midi_path)
  if delta != 0:
    for instrument in midi_data.instruments:
      for note in instrument.notes:
        note.velocity += delta
    midi_data.write(output_path)

def convert_midi_to_wav(soundfont_path, midi_path, output_path, gain=None, velocity_change=0): # Renamed the argument
  change_midi_velocity(midi_path, "temp.mid", delta=velocity_change) # Call the renamed function
  FluidSynth(soundfont_path, gain=gain).midi_to_audio("temp.mid", output_path)
  os.remove("temp.mid")


def trim_midi(midi_path, start, end):
  import mido
  import midi_clip
  mid = mido.MidiFile(midi_path)
  trimmed_midi = midi_clip.midi_clip(mid, start, end)

  dir_name, base_name = os.path.split(midi_path)
  new_base_name = "trimmed_" + base_name
  output_path = os.path.join(dir_name, new_base_name)
  trimmed_midi.save(output_path)
  return output_path

def playMidi(midi_file_path,
             soundfont_path="/content/guGS.sf2",
             output_path="audio.wav",
             start=None,
             end=None,
             gain=DEFAULT_GAIN,
             velocity_change=0
             ):
    from IPython.display import Audio

    if start is not None and end is not None:
      midi_file_path = trim_midi(midi_file_path, start, end)
      convert_midi_to_wav(soundfont_path, midi_file_path, output_path, gain=gain, velocity_change=velocity_change)
      os.remove(midi_file_path)
    else:
      convert_midi_to_wav(soundfont_path, midi_file_path, output_path, gain=gain, velocity_change=velocity_change)
    return Audio(output_path)

In [25]:
import matplotlib.pyplot as plt

path = random_file(dataset_path)
print("Converting: " + path)
print("Midi info:")
show_midi_info(path)

Converting: /content/clean_midi/Londonbeat/I've Been Thinking About You.1.mid
Midi info:
Instruments:  ['ACOU BASS', 'E.PIANO 2', 'STEEL GTR', 'STRINGS', 'MELODY', 'BASS DRUM', 'SNARE DRUM', 'CLOSED H.H', 'HANDCLAP', 'HIGH TOM', 'MID TOM', 'LOW TOM', 'CRASH CYMB']
MIDI duration: 224.11 seconds


## Play a random MIDI of the Lakh dataset

In [63]:
path = random_file(dataset_path)
print("Converting: " + path)
print("Midi info:")
show_midi_info(path)
print("Synthetized:")
playMidi(path)

Converting: /content/clean_midi/Scorpions/Send Me an Angel.mid
Midi info:
Instruments:  ['Melodie', 'Strings', 'Choir', 'Akk-Guita', 'Solo-Guit', 'Solovox', 'Bass', 'Drums']
MIDI duration: 268.73 seconds
Synthetized:


/usr/local/lib/python3.10/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: 'temp.mid'

## Play a random MIDI of the NESMDB dataset

In [ ]:
path = random_file("nesmdb_midi")
print("Converting: " + path)
print("Midi info:")
show_midi_info(path)
print("Synthetized:")
playMidi(path, soundfont_path="PICONICA.sf2", velocity_change=30, gain=1)

In [64]:
show_midi_info("generated_midi_0.mid")

Instruments:  ['']
MIDI duration: 1280.00 seconds


In [65]:
playMidi("generated_midi_0.mid", soundfont_path="PICONICA.sf2")

FileNotFoundError: [Errno 2] No such file or directory: 'temp.mid'